In [ ]:
# https://wikidocs.net/233348

In [ ]:
# pip install langchain

In [ ]:
# pip install -U langchain-ollama

In [ ]:
# pip install langchain_teddynote

In [1]:
import warnings
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.messages import stream_response
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore", category=DeprecationWarning)
# https://wikidocs.net/233348

In [2]:
df = pd.read_csv('../../data/241113_llama_ver1.csv', encoding='cp949')

In [3]:
llm = OllamaLLM(model="llama-3.1-70b-instruct-lorablated.Q4_K_M:latest")

In [4]:
df.shape[0]

44

In [5]:
population = np.arange(44)
np.random.seed(1111)
first_ann_q_id = np.random.choice(population, size=4, replace=False)


In [6]:
first_ann_q_id

array([ 6, 23, 41, 25])

In [7]:
df.iloc[0, 0]

'Here is the question about software development. The title of the question is "Why is this gunicorn, flask, bokeh application not running with 4 workers?". and body of the question is "<p>I am learning to use gunicorn, flask and bokeh.\nI am trying to set up a little example to use a bokeh app with multiuser. I am working on a debian system on an AWS server with a static IP. I used the <a href="https://github.com/bokeh/bokeh/blob/2.4.3/examples/howto/server_embed/flask_gunicorn_embed.py" rel="nofollow noreferrer">example from git</a> and edited it in the way I think it should work. I got it to run with this command:</p>\n<pre><code>gunicorn3 -w 1 --bind 0.0.0.0:8000 wsgi:app\n</code></pre>\n<p>But when I change the workers to 4 and the binded port from 5006 to 0 (flaskapp.py), I get this error in the browser console:</p>\n<blockquote>\n<p>:34163/bkapp/autoload.js?bokeh-autoload-element=4755&amp;bokeh-app-path=/bkapp&amp;bokeh-absolute-url=http://35.XXX.XXX.153:34163/bkapp:1          F

In [19]:
examples = []
for idx in first_ann_q_id[:3]:
    temp_dict = {"question" : df.iloc[idx, 0],
                 "answer"   : df.iloc[idx, 1]}
    examples.append(temp_dict)

In [20]:
examples[0]

{'question': 'Here is the question about software development. The title of the question is "Strange error: TypeError: forward() takes 2 positional arguments but 3 were given". and body of the question is "<p>I am trying to train a VAE model. However I keep getting the following error:</p>\n<pre><code>Traceback (most recent call last):\n  File &quot;/Users/devcharaf/Documents/Uni/UdeM/Internship/newGNN/app/train.py&quot;, line 28, in &lt;module&gt;\n    trained_model, loss = train(model, train_data, optimizer, num_epochs=1000, model_type=&quot;VAE&quot;)\n  File &quot;/Users/devcharaf/Documents/Uni/UdeM/Internship/newGNN/app/utils.py&quot;, line 322, in train\n    recon_x, mean, logvar = model(data[&quot;x&quot;], data[&quot;edge_index&quot;])\n  File &quot;/opt/anaconda3/envs/pygeometric/lib/python3.9/site-packages/torch/nn/modules/module.py&quot;, line 1194, in _call_impl\n    return forward_call(*input, **kwargs)\n  File &quot;/Users/devcharaf/Documents/Uni/UdeM/Internship/newGNN/ap

In [22]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question:
Here is the question about software development. The title of the question is "Strange error: TypeError: forward() takes 2 positional arguments but 3 were given". and body of the question is "<p>I am trying to train a VAE model. However I keep getting the following error:</p>
<pre><code>Traceback (most recent call last):
  File &quot;/Users/devcharaf/Documents/Uni/UdeM/Internship/newGNN/app/train.py&quot;, line 28, in &lt;module&gt;
    trained_model, loss = train(model, train_data, optimizer, num_epochs=1000, model_type=&quot;VAE&quot;)
  File &quot;/Users/devcharaf/Documents/Uni/UdeM/Internship/newGNN/app/utils.py&quot;, line 322, in train
    recon_x, mean, logvar = model(data[&quot;x&quot;], data[&quot;edge_index&quot;])
  File &quot;/opt/anaconda3/envs/pygeometric/lib/python3.9/site-packages/torch/nn/modules/module.py&quot;, line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File &quot;/Users/devcharaf/Documents/Uni/UdeM/Internship/newGNN/app/model.py&q

In [24]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

In [25]:
prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'Here is the question about software development. The title of the question is "Strange error: TypeError: forward() takes 2 positional arguments but 3 were given". and body of the question is "<p>I am trying to train a VAE model. However I keep getting the following error:</p>\n<pre><code>Traceback (most recent call last):\n  File &quot;/Users/devcharaf/Documents/Uni/UdeM/Internship/newGNN/app/train.py&quot;, line 28, in &lt;module&gt;\n    trained_model, loss = train(model, train_data, optimizer, num_epochs=1000, model_type=&quot;VAE&quot;)\n  File &quot;/Users/devcharaf/Documents/Uni/UdeM/Internship/newGNN/app/utils.py&quot;, line 322, in train\n    recon_x, mean, logvar = model(data[&quot;x&quot;], data[&quot;edge_index&quot;])\n  File &quot;/opt/anaconda3/envs/pygeometric/lib/python3.9/site-packages/torch/nn/modules/module.py&quot;, line 1194, in _call_impl\n    return f

In [26]:
print(df.shape)
print(first_ann_q_id[3])

(44, 2)
25


In [27]:
type(df.iloc[first_ann_q_id[3], 0])

str

In [37]:
prompt = " {question}"  # Example prompt
question = df.iloc[first_ann_q_id[3], 0]

# Format the prompt with the question
try:
    final_prompt = prompt.format(question=question)
    print(final_prompt)
except KeyError as e:
    print(f"Missing placeholder in prompt: {e}")
except IndexError as e:
    print(f"Index error in prompt formatting: {e}")

 Here is the question about software development. The title of the question is "Add error handling in Python API call for Meraki". and body of the question is "<p>I have this script to make an API call using the meraki python module to query a Meraki device.</p>
<p>I'm trying to add some error handling in the code so that if the API call comes back with an error code, it will do something else. I cannot seem to figure out what to do.</p>
<p>Here is my simple code to just query a device:</p>
<pre><code>import meraki
import requests

API_KEY = 'API_KEY'
dashboard = meraki.DashboardAPI(API_KEY)

serial = input(&quot;What is the serial number?&quot;)
print(f&quot;{serial}&quot;)

response = (dashboard.devices.getDevice(serial))
</code></pre>
<p>When I run the script it will return either a &quot;200 OK&quot; or &quot;404 Not Found&quot;</p>
<p>Terminal window response:</p>
<pre class="lang-none prettyprint-override"><code>C:\Scripts\Meraki\dev&gt; python .\getdevice.py
What is the serial n

In [38]:
# 질의
answer = llm.stream(question)
stream_response(answer)

Based on the provided flowchart, I'll analyze the difficulty level of the question.

The question is about adding error handling in a Python API call. Following the decision path:

1. Is it about basic syntax/environment? No
2. Is it about performance/optimization? No
3. Is it about frameworks? Yes (Meraki API)

So, the category code is I3 (Intermediate Level Knowledge).

The required knowledge for this question includes:

* Web frameworks (K6)
* Error handling ( implicit in K3, but not explicitly listed; however, error handling is a fundamental concept that can be applied at various levels)

Therefore, the difficulty level and required knowledge for this question are:

[Intermediate Level Knowledge] | [I3] | [Web frameworks/Error handling]

**Difficulty Class:** Intermediate

The question requires understanding of web frameworks (Meraki API) and error handling concepts. The solution involves applying these concepts to a specific problem, which is characteristic of intermediate-level q